# 图像基本操作

- 访问像素值并修改它们
- 访问图像属性
- 设置感兴趣区域（ROI）
- 拆分和合并图像

In [2]:
cd C:\Users\Administrator\Desktop

C:\Users\Administrator\Desktop


In [4]:
import numpy as np
import cv2

img = cv2.imread('test.jpg')

## 获取并修改像素值

In [5]:
#  it returns an array of Blue, Green, Red values
img[100, 100]

array([199, 227, 210], dtype=uint8)

In [7]:
# For grayscale image, just corresponding intensity is returned
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray[100, 100]

219

In [10]:
img[101, 101] = [255, 255, 255]
img[101, 101]

array([255, 255, 255], dtype=uint8)

numpy是经过优化了的进行快速矩阵运算的包，所以不推荐逐个获取像素值并修改能矩阵运算就不要用循环。

例如前5行的后3列，用numpy的array.item()和array.itemset()会更好。但是返回是标量，如果想获得所有RGB的值，需要使用array.item()分割他们。
更好的方法是

In [11]:
import cv2
import numpy

img = cv2.imread('test.jpg')
print(img.item(10, 10, 2))
img.itemset((10, 10, 2), 100)
print(img.item(10, 10, 2))

210
100


## 获取图像属性
图像属性包括：行，列，通道，图像数据类型，像素数目等

In [12]:
import cv2
import numpy

img = cv2.imread('test.jpg')
print(img.shape)

(472, 643, 3)


In [13]:
# 返回图像的像素数目
print(img.size)

910488


In [14]:
# 数据类型
print(img.dtype)

uint8


## 图像ROI
对图像的特定区域操作。ROI是使用numpy索引来获得的

In [5]:
import cv2
import numpy

img = cv2.imread('roi.jpg')
print(img.shape)

ball = img[180:210, 30:60]
img[120:150, 90:120] = ball

(212, 347, 3)


## 拆分及合并图像通道

警告：

cv2.split 时间消耗较大，需要时尽可能选择 Numpy 的索引操作会更快

In [7]:
b, g, r = cv2.split(img)  #拆分
b

array([[240, 228, 250, ..., 251, 220, 250],
       [239,  55,  32, ...,  52,  78, 233],
       [246,  29,  28, ...,  38,  50, 250],
       ...,
       [212,  74,  71, ...,  82,  93, 231],
       [210,  72,  68, ...,  80,  91, 230],
       [210,  72,  67, ...,  77,  89, 229]], dtype=uint8)

In [14]:
img = cv2.merge((b, g, r))  #合并

或者

In [6]:
b = img[:, :, 0]
b

array([[240, 228, 250, ..., 251, 220, 250],
       [239,  55,  32, ...,  52,  78, 233],
       [246,  29,  28, ...,  38,  50, 250],
       ...,
       [212,  74,  71, ...,  82,  93, 231],
       [210,  72,  68, ...,  80,  91, 230],
       [210,  72,  67, ...,  77,  89, 229]], dtype=uint8)

## 为图像扩边（填充）

想为图像周围建一个边可以使用**cv2.copyMakeBorder()**函数。这经常在**卷积运算或0填充时**被用到。具体参数如下：
- 5.1 src输入图像
- top,bottom,left,right对应边界的像素数目
- borderType要添加哪种类型的边界：
    - cv2.BORDER_CONSTANT添加有颜色的常数值边界，还需要下一个参数（value）
    - cv2.BORDER_REFLIECT边界元素的镜像。例如：fedcba | abcdefgh | hgfedcb
    - cv2.BORDER_101或者cv2.BORDER_DEFAULT跟上面一样，但稍作改动，例如：gfedcb | abcdefgh | gfedcba
    - cv2.BORDER_REPLICATE重复最后一个元素。例如: aaaaaa| abcdefgh|hhhhhhh
    - cv2.BORDER_WRAP 不知怎么了, 就像样: cdefgh| abcdefgh|abcdefg
- value边界颜色

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

BLUE = [255, 0, 0]
img1 = cv.imread('rio.jpg')
replicate = cv.copyMakeBorder(img1, 10, 10, 10, 10, cv.BORDER_REPLICATE)
reflect = cv.copyMakeBorder(img1, 10, 10, 10, 10, cv.BORDER_REFLECT)
reflect101 = cv.copyMakeBorder(img1, 10, 10, 10, 10, cv.BORDER_REFLECT_101)
wrap = cv.copyMakeBorder(img1, 10, 10, 10, 10, cv.BORDER_WRAP)
constant = cv.copyMakeBorder(
    img1, 10, 10, 10, 10, cv.BORDER_CONSTANT, value=BLUE)

In [ ]:
plt.subplot(231), plt.imshow(img1, 'gray'), plt.title('ORIGINAL')
plt.subplot(232), plt.imshow(replicate, 'gray'), plt.title('REPLICATE')
plt.subplot(233), plt.imshow(reflect, 'gray'), plt.title('REFLECT')
plt.subplot(234), plt.imshow(reflect101, 'gray'), plt.title('REFLECT_101')
plt.subplot(235), plt.imshow(wrap, 'gray'), plt.title('WRAP')
plt.subplot(236), plt.imshow(constant, 'gray'), plt.title('CONSTANT')
plt.show()

# 图像上的算术运算

## 图像加法
使用cv2.add()将两幅图像进行加法运算，也可以直接使用numpy，res=img1+img2.两幅图像的大小，类型必须一致，或者第二个图像可以是一个简单的标量值。

In [2]:
import cv2
import numpy as np

x=np.uint8([250])
y=np.uint8([10])

print(cv2.add(x,y))#250+10=260>=255

[[255]]


In [3]:
print (x+y)#250+10=260%255=4

[4]


OpenCV的结果会更好，so尽量使用OpenCV中的函数

## 图像混合
这也是加法，不同的是两幅图像的权重不同，这会给人一种混合或者透明的感觉。图像混合的计算公式如下：

g(x) = (1−α)f0 (x)+αf1 (x)

通过修改α的值（0-->1）,可以实现很酷的混合。

例：将两幅图像混合，第一幅权重为0.7.第二幅权重为0.3。函数cv2.addWeighed()可以按下面的公式对图片进行混合。

dst = α·img1 + β·img2+γ

这里γ的取值为0.

```python
import cv2
import numpy as np
img1 = cv2.imread('45.jpg')
img2 = cv2.imread('messigray.png')

dst = cv2.addWeighted(img1, 0.7, img2, 0.3, 0)

cv2.imshow('dst', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()
```
![](https://docs.opencv.org/4.1.0/blending.jpg)

## 按位运算
这里包括按位操作有：AND，OR，NOT，XOR等，当我们提取图像的一部分，选择非矩形ROI时，会很有用（下章）。下面进行如何改变一幅图的特定区域。

In [ ]:
import cv2
import numpy as np

img1 = cv2.imread('45.jpg')
img2 = cv2.imread('messigray.png')

# I want to put logo on top-left corner, So I create a ROI
rows, cols, channels = img2.shape
roi = img1[0:rows, 0:cols]

# Now create a mask of logo and create its inverse mask also
img2gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
# 二值化
ret, mask = cv2.threshold(img2gray, 175, 255, cv2.THRESH_BINARY)
mask_inv = cv2.bitwise_not(mask)

# Now black-out the area of logo in ROI
#取ROI中与mask中不为零的值对应的像素的值，其让值为0 。
#注意这里必须有mask=mask或者mask=mask_inv，其中mask=不能忽略
img1_bg = cv2.bitwise_and(roi, roi, mask=mask)

#取roi中与mask_inv中不为零的值对应的像素的值，其他值为0
# Take only region of logo from logo image.
img2_fg = cv2.bitwise_and(img2, img2, mask=mask_inv)

# Put logo in ROI and modify the main image
dst = cv2.add(img1_bg, img2_fg)
img1[0:rows, 0:cols] = dst

cv2.imshow('res', img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

![](https://docs.opencv.org/4.1.0/overlay.jpg)

# 程序性能检测及优化

## 使用OpenCV检测程序效率

涉及函数 **cv2.getTickCount，cv2.getTickFrequency**

1.使用OpenCV检测程序效率
cv2.getTickCount函数返回从参考点到这个函数被执行的时钟数。在一个函数执行前后都调用它，可以得到这个函数的执行时间。

cv2.getTickFrequency返回时钟频率，或者说每秒钟的时钟数
例，窗口大小不同（5,7,9）的核函数来做中值滤波，查看一个函数运行了多少秒

In [17]:
import cv2
import numpy as np

img = cv2.imread('roi.jpg')

e1 = cv2.getTickCount()
for i in range(5, 49, 2):
    img = cv2.medianBlur(img, i)
e2 = cv2.getTickCount()
time = (e2 - e1) / cv2.getTickFrequency()
print(time)

0.00020842450332101035


In [10]:
cv2.getTickFrequency()

2648441.0

## OpenCV中的默认优化
cv2.useOptimized()来查看优化是否被开启，cv2.setUesOptimized()来开启优化。

In [18]:
import cv2 
import numpy as np

# check if optimization is enabled 
cv2.useOptimized()

True

In [19]:
%timeit res = cv2.medianBlur(img, 49) 

729 ns ± 107 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [20]:
# Disable it 
cv2.setUseOptimized(False)

In [21]:
cv2.useOptimized()

False

In [22]:
%timeit res = cv2.medianBlur(img,49)

782 ns ± 77.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## 在IPython中检测程序效率

In [ ]:
import cv2
import numpy as np

x =5

In [23]:
%timeit y=x**2 

418 ns ± 47.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [26]:
%timeit y=x*x 

67.2 ns ± 6.29 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [24]:
z = np.uint8([5])

%timeit y=z*z 

646 ns ± 60.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [25]:
%timeit y=np.square(z)

621 ns ± 53.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


**注意**：Python 的标算比 Nump 的标算快。对于仅包含一两个 元素的操作 Python 标比 Numpy 的数组快。但是当数组稍微大一点时 Numpy 就会胜出了

In [28]:
%timeit z = cv2.countNonZero(img)

622 ns ± 101 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [29]:
%timeit z = np.count_nonzero(img)

1.25 µs ± 78.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


See, OpenCV function is nearly 25x faster than Numpy function.

## 效率优化技术
有些技术和编程方法可以我们大的发挥 Python 和 Numpy 的威力。 我们仅仅提一下相关的你可以接查找更多细信息。我们 的的一点是先用简单的方式实现你的算法结果正确当结 果正确后再使用上的提到的方法找到程序的瓶来优化它。

- 尽免使用循环尤其双层三层循环它们天生就是常慢的。
- 算法中尽使用向操作因为 Numpy 和 OpenCV 对向操作 了优化。
- 利用缓存一致性。
- 没有必的就不复制数组。使用图来代替复制。数组复制是常浪源的。
就算了上优化如果你的程序是很慢或者大的不可免的 你你应尝使用其他的包比如 Cython来加你的程序。

1. [Python Optimization Techniques](http://wiki.python.org/moin/PythonSpeed/PerformanceTips)
2. Scipy Lecture Notes - [Advanced Numpy](http://scipy-lectures.github.io/advanced/advanced_numpy/index.html#advanced-numpy)